In [3]:
import torch
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer



In [4]:
data = pd.read_csv('translated_sentences(1).csv')

In [5]:
data.head()

,Original Hindi Sentences,Translated Sentences
0,['अनुसंधानकर्ताओं को चाहिए कि वे किसी प्रकार क...,"वे इस खतरे को नहीं लेना चाहेंगे, खासकर इसलिए क..."
1,['कुछ प्रकार के शस्त्रक्रिया में शामिल है शरीर...,कुछ तरह के ऑपरेशन में शरीर-LBB-B के एक हिस्से ...
2,['इसके मुख़्य लक्षण तो हैं : तापमान का बढ़ जान...,"इसके मुख्य लक्षण तापमान में उठते हैं, और वुल्फ..."
3,['ऐसी स्थिति में उनका सामुदायिक विकास कार्यक्र...,उसका समुदाय विकास का कार्यक्रम इसलिए स्व-help ...
4,"[""यह बात सर जिओफ्रे दि मोंटमोरंसी नाम के एक मश...",इस तथ्य पर हाल ही में एक ब्रिटिश अधिकारी ने ज़...


In [6]:
t5_checkpoint = "csebuetnlp/mT5_multilingual_XLSum"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer_t5 = AutoTokenizer.from_pretrained(t5_checkpoint)
model_t5 = AutoModelForSeq2SeqLM.from_pretrained(t5_checkpoint).to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
c:\Users\mihik\anaconda3\lib\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [7]:
lower_limit = 0
upper_limit = 100

In [8]:
text = data.loc[lower_limit:upper_limit, "Translated Sentences"]
original = data.loc[lower_limit:upper_limit, "Original Hindi Sentences"]

In [9]:
%%time

SystemSummary = []

for i, input_text in enumerate(text):    
    
    input_ids = tokenizer_t5(input_text,
                             return_tensors="pt",
                             padding="max_length",
                             truncation=True,
                             max_length=768)["input_ids"].to(device) 

    model_output = model_t5.generate(input_ids=input_ids,
                                     num_beams=4,
                                     max_length=70,
                                     min_length=30,
                                     no_repeat_ngram_size=2,
                                     early_stopping=True)

    decoded_output = tokenizer_t5.decode(model_output[0],
                                  skip_special_tokens=True,
                                  clean_up_tokenization_spaces=False)
    
    SystemSummary.append(decoded_output)

CPU times: total: 1h 48min 49s
Wall time: 23min 55s


In [10]:
columns = {'Original_sentence':original 'translated_sentence':text, 'predicted_summaries': SystemSummary}

SyntaxError: invalid syntax (838196885.py, line 1)

In [ ]:
df = pd.DataFrame(columns)

In [ ]:
df.to_csv('summaries' + upper_limit+ '.csv')